## Importing packages

In [1]:
import pandas as pd

## New York Business Dataset - cleaning
Note: If the full datasets are taking too long to run, you could use the smaller 10-core subsets for experimenting.

In [3]:
# Specify the file path
file_path = 'Raw Data/review-New_York.json'

# Set the chunk size to an appropriate value based on your available memory
chunk_size = 10000  # Adjust this value as needed

# Initialize an empty list to store the dataframes
dfs = []

# Use a loop to read the file in chunks
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    dfs.append(chunk)

# Concatenate the list of dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Remove identifiable names | axis 1 removes by column
df = df.drop('name', axis=1)

print("The DataFrame :")
display(df.head())

The DataFrame :


,user_id,time,rating,text,pics,resp,gmap_id
0,1.018558e+20,1629141186463,1.0,Natalia may be the worst agent I have ever dea...,None,None,0x89c24469c758686b:0x641f5b84cb9bedfa
1,1.058219e+20,1528477593994,1.0,The lady at the front desk is rude. The bathro...,None,None,0x89c24469c758686b:0x641f5b84cb9bedfa
2,1.089909e+20,1424830512308,1.0,"Worst agent in New York and Brooklyn, dont was...",None,None,0x89c24469c758686b:0x641f5b84cb9bedfa
3,1.170215e+20,1512641660497,5.0,None,None,None,0x89c24469c758686b:0x641f5b84cb9bedfa
4,1.137221e+20,1603494795361,5.0,I'm late to posting this but this store especi...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c25fc9494dce47:0x6d63c807b59a55


## Filter business dataset based on metadata 'category'
Currently, this section filters businesses to only include rows where "restaurant" is mentioned in the business metadata category. We should further explore the metadata in the full dataset to understand how 'category' is structured. Perhaps there are other sub-categories which are being accidentally excluded (e.g., steakhouse, food court, cafe, coffee shop, etc.)

In [5]:
# Convert json to dataframe
metadata = pd.read_json('Raw Data/meta-New_York.json',lines=True)
display(metadata.head())

# Flatten the lists in the "category" column and convert them to lowercase
metadata['category'] = metadata['category'].apply(lambda x: [item.lower() for item in x] if isinstance(x, list) else [])

# Filter the metadata DataFrame to create a list of business IDs with "restaurant" in the category
restaurant_ids = metadata[metadata['category'].apply(lambda x: 'restaurant' in x)]['gmap_id'].tolist()


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,A-Top Insurance,"A-Top Insurance, 1009 Brighton Beach Ave, Broo...",0x89c24469c758686b:0x641f5b84cb9bedfa,None,40.578254,-73.959127,"[Insurance broker, Insurance agency]",2.0,4,None,"[[Thursday, 10AM–6PM], [Friday, 10AM–6PM], [Sa...",None,Open ⋅ Closes 6PM,"[0x89c24449907718fb:0x31b554a0983f621d, 0x4065...",https://www.google.com/maps/place//data=!4m2!3...
1,T-Mobile,"T-Mobile, 3923 103rd St, Queens, NY 11368",0x89c25fc9494dce47:0x6d63c807b59a55,None,40.750146,-73.862536,"[Cell phone store, Electronic parts supplier, ...",3.5,95,$$,"[[Thursday, 10AM–8PM], [Friday, 10AM–8PM], [Sa...","{'Service options': ['In-store shopping', 'Del...",Open ⋅ Closes 8PM,"[0x89c25fc7a91c609f:0xb103d6a261373fd, 0x89c25...",https://www.google.com/maps/place//data=!4m2!3...
2,Ace Hardware,"Ace Hardware, 130 4th Ave, New York, NY 10003",0x89c259992463b6e1:0x4a601a43752541dd,"Chain retailer stocking household tools, suppl...",40.733182,-73.990201,"[Hardware store, Building materials store, Ele...",3.4,8,None,"[[Thursday, 8AM–6:30PM], [Friday, 8AM–6:30PM],...",None,Permanently closed,"[0x89c2598340fb6509:0xa996f3db49ba6e78, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...
3,"Agricultural Consulting Services, Inc.","Agricultural Consulting Services, Inc., 370 Su...",0x89d14bf74320aea5:0x720261cd8960f8cd,None,43.054743,-77.649136,[Corporate office],3.0,1,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",None,Open ⋅ Closes 5PM,None,https://www.google.com/maps/place//data=!4m2!3...
4,Long Point,"Long Point, Ledyard, NY 13026",0x89d0bd8d2df8d10f:0x77a8f349f40492fa,None,42.715626,-76.710227,[Peninsula],5.0,1,None,None,None,None,None,"https://www.google.com/maps/place/Ledyard,+NY+..."


In [6]:
# Filter the review dataframe to include only business IDs that appear in the list of restaurant IDs
restaurantreviews = df[df['gmap_id'].isin(restaurant_ids)]
display(restaurantreviews.head())

# Filter the metadata dataframe to include only businesses that appear in the list of restaurant IDs
restaurantmetadata = metadata[metadata['gmap_id'].isin(restaurant_ids)]


,user_id,time,rating,text,pics,resp,gmap_id
652,1.094350e+20,1384783434547,5.0,I came by yesterday to pick up some pizza on t...,None,None,0x89c258ffaeaba947:0x8355860772a595a9
653,1.032679e+20,1415746051592,5.0,Great place. They had 4 soups self serve so y...,None,None,0x89c258ffaeaba947:0x8355860772a595a9
654,1.085845e+20,1424212048498,1.0,Food was burnt and not good. You would think s...,None,None,0x89c258ffaeaba947:0x8355860772a595a9
655,1.089870e+20,1419979427508,1.0,This place is horrible. I ordered two ziti piz...,None,None,0x89c258ffaeaba947:0x8355860772a595a9
656,1.007163e+20,1402593742697,1.0,Ordered a minestrone soup and got some water w...,None,None,0x89c258ffaeaba947:0x8355860772a595a9


### Convert date/time
Note: This assumes that the review timestamps are UNIX format in milliseconds. Reviews should only range from early 2000s to 2021, earlier review timestamps indicate some sort of error.

In [7]:
restaurantreviews['time'] = pd.to_datetime(restaurantreviews['time'], unit='ms')

C:\Users\sammh\AppData\Local\Temp\ipykernel_1808\1014142260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurantreviews['time'] = pd.to_datetime(restaurantreviews['time'], unit='ms')


In [8]:
# Find the earliest date
earliest_date = restaurantreviews['time'].min()

# Find the latest date
latest_date = restaurantreviews['time'].max()

# Print the results
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 1990-12-31 00:00:00
Latest Date: 2021-09-08 19:40:17.953000


## Count occurrences of category types

In [9]:
# Flatten the list of categories
categories_flat = [category for categories in metadata['category'] for category in categories]

# Count unique occurrences
category_counts = pd.Series(categories_flat).value_counts()

# Create a DataFrame from the counts
counts_df = pd.DataFrame({'category': category_counts.index, 'count': category_counts.values})

# Save to CSV
counts_df.to_csv('category_counts.csv', index=False)

### Saving cleaned datasets to CSV files
Could still do some more work to make these datasets even cleaner!

In [15]:
# Save the filtered restaurant reviews and metadata to a CSV file
restaurantreviews.to_csv('NYCrestaurantreviews.csv', index=False)
restaurantmetadata.to_csv('NYCrestaurantmetadata.csv', index=False)

## Exploring noise keywords
Just experimenting with some possible keywords of interest.

In [17]:
noise_keywords = ["loud", "noisy", "noise", "sound", "quiet", "can't hear", "hard to hear", "cannot hear"]

# Count rows containing noise keywords in the "text" column
count_noise_rows = restaurantreviews['text'].str.contains('|'.join(noise_keywords), case=False, na=False).sum()
count_all_rows = len(restaurantreviews.index)

# Print the count
print(f"Number of rows containing noise-related keywords: {count_noise_rows} out of {count_all_rows}")

Number of rows containing noise-related keywords: 43681 out of 5877719
